
<p align="center"><img src="https://github.com/Reece323/air-scrape/blob/main/assets/airbnb_logo.png?raw=true" width="300"/></p>
<center> <h1> Analyzing AirBnB Listings </h1> </center>



### This project focuses on data gathered from AirBnB listings
#### Goals:


*   *Manipulating and Exploring the data*
*   *Creating Visualizations*
*   *Use Machine Learning*
*   *Determine which features attracts consumers*
*   *See how to increase revenue as `host`*

In [122]:
# Imports
import pandas as pd
import numpy as np
import json
import urllib.request
from operator import itemgetter
from pprint import pprint
from itertools import chain
import re

# !pip install rich
from rich.console import Console

console = Console()
print = console.print

pd.options.display.max_columns = 4000

import folium
import contextily as cx
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point
from geopandas import GeoDataFrame
import matplotlib as mpl
from matplotlib import cm
from colorspacious import cspace_converter

In [113]:
url = 'https://raw.githubusercontent.com/Reece323/air-scrape/main/Data/Bentonville.json'
response = urllib.request.urlopen(url)
Bentonville = json.loads(response.read())
Bville = pd.json_normalize(Bentonville)

In [114]:
cols_to_drop_temp = [
    'url',
    "calendar",
    'rootAmenitySections',
    'p3SummaryAddress',
    'photos',
    'primaryHost.about',
    'primaryHost.badges',
    'primaryHost.firstName',
    'primaryHost.id',
    'primaryHost.isSuperhost',
    'primaryHost.memberSinceFullStr',
    'primaryHost.name',
    'primaryHost.languages',
    'primaryHost.pictureUrl',
    'primaryHost.responseRateWithoutNa',
    'primaryHost.responseTimeWithoutNa',
    'primaryHost.hasInclusionBadge',
    'primaryHost.pictureLargeUrl',
    'primaryHost.hostIntroTags',
    'primaryHost.hostUrl',
    'primaryHost.listingsCount',
    'primaryHost.totalListingsCount',
    'tierId',
    'hasSpecialOffer',
    'hasWeWorkLocation',
    'hasWeWorkLocation',
    'additionalHosts',
    'hometourRooms',
    'hometourSections',
    'descriptionLocale',
    'initialDescriptionAuthorType',
    'localizedCheckInTimeWindow',
    'localizedCheckOutTime',
    'hometourRooms',
    'countryCode',
    'hasHostGuidebook',
    'hasLocalAttractions',
    'neighborhoodCommunityTags',
    'state',
    'paidGrowthRemarketingListingIds',
    'hasCommercialHostInfo',
    'hostSignatureFont.url',
    'nearbyAirportDistanceDescriptions',
    'propertyTypeInCity',
    'renderTierId',
    'isHotel',
    'isNewListing',
    'showReviewTag',
    'isRepresentativeInventory',
    'localizedCity',
    'highlights',
    'highlightsImpressionId',
    'pointOfInterests',
    'hostGuidebook.guidebookUrl',
    'hostGuidebook.localizedNameForHomesPdp',
    'hostGuidebook.title',
    'hostGuidebook.id',
    'pageViewType',
    'previewTags',
    'seeAllHometourSections',
    'enableHighlightsVoting',
    'heroModule.categorizedPhotos',
    'sortedReviews',
    'documentDisplayPictures',
    'sections',
    'p3ImpressionId',
    'paidGrowthRemarketingListingIdsStr',
    'isBusinessTravelReady',
    'address',
    'hasHouseRules',
    'reviewsOrder',
    'hostQuote',
    'localizedListingExpectations',
    'pricing.rate.amount_formatted',
    'pricing.rate.currency',
    'pricing.rate.is_micros_accuracy',
    'pricing.rate_with_service_fee.amount_formatted',
    'pricing.rate_with_service_fee.currency',
    'pricing.rate_with_service_fee.is_micros_accuracy',
    'reviewDetailsInterface.reviewSummary',
    'reviewsModule.appreciationTags',
    'pricing.rate_with_service_fee.amount',
    'pricing.rate_type',
    'sectionedDescription.authorType',
    'sectionedDescription.locale',
    'sectionedDescription.localizedLanguageName',
    'sectionedDescription.notes',
    'sectionedDescription.space',
    'guestControls.allowsNonChinaUsers',
    'guestControls.structuredHouseRulesWithTips',
    'sectionedDescription.neighborhoodOverview',
    'sectionedDescription.access',
    'sectionedDescription.description',
    'sectionedDescription.interaction',
    'sectionedDescription.name',
    'sectionedDescription.summary',
    'sectionedDescription.transit',
    'name',
    'listingExpectations',
    'sectionedDescription.houseRules',
    'additionalHouseRules',
    'idStr'
]

Bville = Bville.drop(cols_to_drop_temp, axis=1)

In [115]:
df = Bville.reindex(columns=[
    'city',
    'country',
    'id',
    'location.lat',
    'location.lng',
    'numberOfGuests',
    'listingRooms',
    'roomType',
    'roomTypeCategory',
    'bedroomLabel',
    'bedLabel',
    'bathroomLabel',
    'listingAmenities',
    'seeAllAmenitySections',
    'categorizedPreviewAmenities',
    'stars',
    'reviewsModule.localizedOverallRating',
    'reviewDetailsInterface.reviewCount',
    'reviews',
    'isHostedBySuperhost',
    'maxNights',
    'minNights',
    'priceDetails',
    'pricing.rate.amount',
    'guestControls.allowsChildren',
    'guestControls.allowsEvents',
    'guestControls.allowsInfants',
    'guestControls.allowsPets',
    'guestControls.allowsSmoking',
    'guestControls.personCapacity',
    'guestControls.structuredHouseRules',
]
)


In [116]:
df.columns = [re.sub("[ ,\$]", "_", re.sub("[.]", "", c)) for c in df.columns]
print(df.columns)
df

Index(['city', 'country', 'id', 'locationlat', 'locationlng', 'numberOfGuests',
       'listingRooms', 'roomType', 'roomTypeCategory', 'bedroomLabel',
       'bedLabel', 'bathroomLabel', 'listingAmenities',
       'seeAllAmenitySections', 'categorizedPreviewAmenities', 'stars',
       'reviewsModulelocalizedOverallRating',
       'reviewDetailsInterfacereviewCount', 'reviews', 'isHostedBySuperhost',
       'maxNights', 'minNights', 'priceDetails', 'pricingrateamount',
       'guestControlsallowsChildren', 'guestControlsallowsEvents',
       'guestControlsallowsInfants', 'guestControlsallowsPets',
       'guestControlsallowsSmoking', 'guestControlspersonCapacity',
       'guestControlsstructuredHouseRules'],
      dtype='object')

,city,country,id,locationlat,locationlng,numberOfGuests,listingRooms,roomType,roomTypeCategory,bedroomLabel,bedLabel,bathroomLabel,listingAmenities,seeAllAmenitySections,categorizedPreviewAmenities,stars,reviewsModulelocalizedOverallRating,reviewDetailsInterfacereviewCount,reviews,isHostedBySuperhost,maxNights,minNights,priceDetails,pricingrateamount,guestControlsallowsChildren,guestControlsallowsEvents,guestControlsallowsInfants,guestControlsallowsPets,guestControlsallowsSmoking,guestControlspersonCapacity,guestControlsstructuredHouseRules
0,Bentonville,United States,32721035,36.40187,-94.22518,4,"[{'beds': [{'id': '37190702/queen_bed', 'quant...",Entire guest suite,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5,4.96,366,"[{'author': {'firstName': 'Zach', 'hasProfileP...",True,29,2,"[{'label': 'Cleaning Fee', 'value': '$20', 'at...",49,True,False,True,False,False,4,"[No pets, No smoking, parties, or events, Chec..."
1,Bentonville,United States,14023239,36.37226,-94.21269,4,"[{'beds': [{'id': '8053552/sofa_bed', 'quantit...",Entire guesthouse,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5,4.97,329,"[{'author': {'firstName': 'Savannah', 'hasProf...",True,14,1,"[{'label': 'Cleaning Fee', 'value': '$50', 'at...",92,True,False,True,False,False,4,"[No pets, No smoking, parties, or events, Chec..."
2,Bentonville,United States,38783656,36.37763,-94.18872,2,"[{'beds': [{'id': '65625913/king_bed', 'quanti...",Private room in residential home,private_room,1 bedroom,1 bed,1 private bath,"[{'description': '', 'id': 8, 'isPresent': Fal...","[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Wifi, ...",5,4.92,239,"[{'author': {'firstName': 'Rachel', 'hasProfil...",True,28,2,"[{'label': 'Cleaning Fee', 'value': '$10', 'at...",56,True,False,False,False,False,2,[Not safe or suitable for infants (under 2 yea...


In [120]:
df['geometry'] = [(xy) for xy in zip(df.locationlat, df.locationlng)] 

In [121]:
df

,city,country,id,locationlat,locationlng,numberOfGuests,listingRooms,roomType,roomTypeCategory,bedroomLabel,bedLabel,bathroomLabel,listingAmenities,seeAllAmenitySections,categorizedPreviewAmenities,stars,reviewsModulelocalizedOverallRating,reviewDetailsInterfacereviewCount,reviews,isHostedBySuperhost,maxNights,minNights,priceDetails,pricingrateamount,guestControlsallowsChildren,guestControlsallowsEvents,guestControlsallowsInfants,guestControlsallowsPets,guestControlsallowsSmoking,guestControlspersonCapacity,guestControlsstructuredHouseRules,geometry
0,Bentonville,United States,32721035,36.40187,-94.22518,4,"[{'beds': [{'id': '37190702/queen_bed', 'quant...",Entire guest suite,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5,4.96,366,"[{'author': {'firstName': 'Zach', 'hasProfileP...",True,29,2,"[{'label': 'Cleaning Fee', 'value': '$20', 'at...",49,True,False,True,False,False,4,"[No pets, No smoking, parties, or events, Chec...","(36.40187, -94.22518)"
1,Bentonville,United States,14023239,36.37226,-94.21269,4,"[{'beds': [{'id': '8053552/sofa_bed', 'quantit...",Entire guesthouse,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5,4.97,329,"[{'author': {'firstName': 'Savannah', 'hasProf...",True,14,1,"[{'label': 'Cleaning Fee', 'value': '$50', 'at...",92,True,False,True,False,False,4,"[No pets, No smoking, parties, or events, Chec...","(36.37226, -94.21269)"
2,Bentonville,United States,38783656,36.37763,-94.18872,2,"[{'beds': [{'id': '65625913/king_bed', 'quanti...",Private room in residential home,private_room,1 bedroom,1 bed,1 private bath,"[{'description': '', 'id': 8, 'isPresent': Fal...","[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Wifi, ...",5,4.92,239,"[{'author': {'firstName': 'Rachel', 'hasProfil...",True,28,2,"[{'label': 'Cleaning Fee', 'value': '$10', 'at...",56,True,False,False,False,False,2,[Not safe or suitable for infants (under 2 yea...,"(36.37763, -94.18872)"


In [142]:
coords = [36.3729, -94.2088]

my_map = folium.Map(location=coords, zoom_start=12)
my_map

In [143]:
for i in df.geometry:
    folium.Marker(i, popup="Airbnb Bitch!").add_to(my_map)

In [144]:
my_map